# Altair Example 5 - Donald Trump Approval Rating

This notebook creates an alternative visualization of the FiveThirtyEight's [How popular/unpopular is Donald Trump](https://projects.fivethirtyeight.com/trump-approval-ratings/).

- The `weight` from FiveThirtyEight's `approval_poll_list.csv` is a reflection of the uncertainty and is encoded as opacity
- The `samplesize` is the number of respondents and is encoded as circle size
- The `adjusted_approve` and `adjusted_disapprove` from each poll is encoded on the X and Y axis respectively.
- The `enddate` is the date of an individual poll's result
- A poll's results are reflective over several days before and after the `enddate`. Each poll results fade in/out as well as shrinks/grows at a static rate over 10 days
- If a poll's `adjusted_approve` is greater than or equal to `adjusted_disapprove`, the color will be `green`, otherwise, it will be `orange`
- The Approval and Disapproval percentages shown are a 3-day rolling average of the approve_estimate and disapprove_estimate from `approval_topline.csv`
- The Approval percentage will be shown as `green` and Disapproval percentage as `orange`
- The Approval percentage will be shown on top if the `approve_estimate` is greater than or equal to `disapprove_estimate`, otherwise, Disapproval percentage will be shown on top
- The Approval percentage and Disapproval percentage font size will grow or shrink proportionally to the gap between their values
- The Trump emoji will show `happy` if `approve_estimate` is greater than or equal to `disapprove_estimate`, `sad` if `disapprove_estimate` is less than 10 points ahead of `approve_estimate`, or `mad` if `disapprove_estimate` is 10 or more points ahead of `approve_estimate`

Justification for design choices:

- I chose an animation because I think it engages the reader by entertaining them with the motion and how things change over time.
- Individual polls that occur near the same time but have higher error (lower weight/opacity) create higher certainty when considered together. To capture this, I'm using opacity and time dilation to show areas of higher certainty and areas of lower certainty making for a clearer image of what the public perception truly is. higher vs lower certainty
![Shows how uncertainty is visualized](images/certainty.png)
- I've included the percentage values (encoded with position, size, and color) and Trump emojis to make the graphic more effective to the reader so they can more quickly grasp the situation without having to look at the moving blobs and axis.
- I've added a dashed line along the diagonal to make the graphic more effective in splitting higher approval vs higher disapproval polls
- In the title, both the number of days into the presidency and the date are effective in gauging when a certain approval/disapproval rating took place and showing how it changes over time

In [1]:
import pandas as pd
import altair as alt
import numpy as np
from datetime import timedelta

# Load Data
#---------------------------------
topline_df = pd.read_csv('datasets/approval_topline.csv')
topline_df['modeldate'] = pd.to_datetime(topline_df['modeldate'], format='%m/%d/%Y')

polls_df = pd.read_csv('datasets/approval_poll_list.csv')
polls_df['startdate'] = pd.to_datetime(polls_df['startdate'], format='%m/%d/%Y')
polls_df['enddate'] = pd.to_datetime(polls_df['enddate'], format='%m/%d/%Y')
polls_df['createddate'] = pd.to_datetime(polls_df['enddate'], format='%m/%d/%Y')

# Clean Data
#---------------------------------
# We'll be using pandas rolling average instead of Altair's since Altair's looks like crap
t_df = topline_df[topline_df['subgroup'] == 'All polls'].copy()
t_df.set_index('modeldate', inplace=True)
t_df.sort_index(inplace=True)
t_df[['approval_roll', 'disapproval_roll']] = t_df.rolling('3d')[['approve_estimate','disapprove_estimate']].mean()
t_df.reset_index(inplace=True)

# Limit the polling data to all polls
p_df = polls_df[(polls_df['subgroup'] == 'All polls')].copy()

# Create a 50% line
t_df['line_50'] = 50

# Setup new fields
p2_df = p_df.set_index('enddate').sort_index().copy()
p2_df['majority'] = p2_df.apply(lambda x: 'Approve' if x['adjusted_approve']>=x['adjusted_disapprove'] else 'Disapprove', axis=1)

# Add new rows for fading and shrinking
p2_df['record'] = 'original'
p2_df_copy = p2_df.copy()
longevity = 10
for l in range(1,longevity+1):
    perc = (longevity+1-l)/(longevity+1)
    sub_df = p2_df_copy.shift(-l, freq='D')
    plu_df = p2_df_copy.shift(+l, freq='D')
    sub_df['record'] = '-' + str(l)
    plu_df['record'] = '+' + str(l)
    sub_df['weight'] = sub_df['weight'] * perc
    sub_df['samplesize'] = sub_df['samplesize'] * perc
    plu_df['weight'] = plu_df['weight'] * perc
    plu_df['samplesize'] = plu_df['samplesize'] * perc
    p2_df = p2_df.append([sub_df, plu_df])

# Setup Text Fields
t2_df = t_df.set_index('modeldate').copy()
t2_df_copy = t2_df.copy()
for l in range(1,longevity+2):
    sub_df = t2_df_copy.loc[[t2_df_copy.index.min()]].shift(-l, freq='D')
    plu_df = t2_df_copy.loc[[t2_df_copy.index.max()]].shift(+l, freq='D')
    t2_df = t2_df.append([sub_df,plu_df])
a = 'approval_roll'
d = 'disapproval_roll'
t2_df['text1a'] = t2_df.apply(lambda x: str(round(x[a])) + '%' if x[a]>=x[d] else str(round(x[d])) + '%', axis=1)
t2_df['text1b'] = t2_df.apply(lambda x: 'Approval' if x[a]>=x[d] else 'Disapproval', axis=1)
t2_df[['text1ax','text1ay','text1bx','text1by']] = 59.5, 76, 60.5, 76
t2_df['text2a'] = t2_df.apply(lambda x: str(round(x[d])) + '%' if x[a]>=x[d] else str(round(x[a])) + '%', axis=1)
t2_df['text2b'] = t2_df.apply(lambda x: 'Disapproval' if x[a]>=x[d] else 'Approval', axis=1)
t2_df[['text2ax','text2ay','text2bx','text2by']] = 59.5, 72, 60.5, 72
vMax = (t2_df[a]-t2_df[d]).abs().max()
vMin, sMax, sMin = -vMax, 30, 20
t2_df['text1s'] = t2_df.apply(lambda x: (abs(x[a]-x[d])-vMin)/(vMax-vMin)*(sMax-sMin)+sMin , axis=1)
t2_df['text2s'] = t2_df.apply(lambda x: (-abs(x[a]-x[d])-vMin)/(vMax-vMin)*(sMax-sMin)+sMin , axis=1)

# Setup Trump emoji
url_hap = 'https://raw.githubusercontent.com/cassova/Altair-Examples---Data-Visualizations/main/asset/t_hap.png'
url_sad = 'https://raw.githubusercontent.com/cassova/Altair-Examples---Data-Visualizations/main/asset/t_sad.png'
url_mad = 'https://raw.githubusercontent.com/cassova/Altair-Examples---Data-Visualizations/main/asset/t_mad.png'
t2_df['imgu'] = t2_df.apply(lambda x: url_hap if x[a]>=x[d] else (url_sad if (x[d]-x[a])<10 else url_mad), axis=1)
t2_df[['imgx','imgy']] = 53,74

In [2]:
# Creates chart of one day's values

def draw_graph(day):
    dt = p2_df.index.min() + timedelta(days=day)
    minX, minY, maxX, maxY = 20, 20, 80, 80

    base = alt.Chart(p2_df.loc[dt]).mark_circle().encode(
        x=alt.X('adjusted_approve:Q', axis=alt.Axis(tickCount=5, title='Approval Rating', titleFontSize=20), scale=alt.Scale(domain=[minX,maxX])),
        y=alt.Y('adjusted_disapprove:Q', axis=alt.Axis(tickCount=5, title='Disapproval Rating', titleFontSize=20), scale=alt.Scale(domain=[minY,maxY])),
        size=alt.Size('samplesize',scale=alt.Scale(domain=[100,500,2000,6000,20000,50000], range=[200,500,3000,8000,15000,30000]), legend=None),
        opacity=alt.Opacity('weight',scale=alt.Scale(domain=[0,1,2,3,4], range=[0.25,0.5,0.75,1.0]), legend=None),
        color=alt.Color('majority',scale=alt.Scale(domain=['Approve','Disapprove'], range=['#009f29','#ff7400']), legend=None),
    )
    line_x_50 = alt.Chart(pd.DataFrame({'x':[50]})).mark_rule().encode(x='x')
    line_y_50 = alt.Chart(pd.DataFrame({'y':[50]})).mark_rule().encode(y='y')
    line_diag = alt.Chart(pd.DataFrame({'x':[minX,maxX],'y':[minY,maxY]})).mark_line(color='black',opacity=0.25, strokeDash=[10,10]).encode(x='x',y='y')

    text1a = alt.Chart(t2_df.loc[[dt]]).mark_text(align='right', size=t2_df.loc[dt,'text1s'], fontStyle='bold').encode(
        x='text1ax:Q', y='text1ay:Q', text='text1a:N',
        color=alt.Color('text1b:N', scale=alt.Scale(domain=['Approval','Disapproval'], range=['#009f29','#ff7400']))
    )
    text1b = alt.Chart(t2_df.loc[[dt]]).mark_text(align='left', size=20, fontStyle='bold').encode(
        x='text1bx:Q', y='text1by:Q', text='text1b:N'
    )
    text2a = alt.Chart(t2_df.loc[[dt]]).mark_text(align='right', size=t2_df.loc[dt,'text2s'], fontStyle='bold').encode(
        x='text2ax:Q', y='text2ay:Q', text='text2a:N',
        color=alt.Color('text2b:N', scale=alt.Scale(domain=['Approval','Disapproval'], range=['#009f29','#ff7400']))
    )
    text2b = alt.Chart(t2_df.loc[[dt]]).mark_text(align='left', size=20, fontStyle='bold').encode(
        x='text2bx:Q', y='text2by:Q', text='text2b:N'
    )
    text = (text1a  + text1b + text2a + text2b)
    
    trump_emoji = alt.Chart(t2_df.loc[[dt]]).mark_image(width=50,height=50).encode(x='imgx',y='imgy',url='imgu')
    
    chart = (base + line_x_50 + line_y_50 + line_diag + text + trump_emoji).properties(
        width=900,
        height=600,
        title=alt.Text(text=f'Day {day} of Donald Trump\'s Presidency',
                       fontSize=30,
                       subtitle=p2_df.loc[dt].reset_index().enddate.dt.strftime('%B %#d, %Y')[0],
                       subtitleFontSize=20
                      )
    )
    return chart

In [3]:
# This is the ipywidget Version (choppy and not very nice...)
import ipywidgets as widgets
from ipywidgets import interact

interact(draw_graph, day = widgets.Play(
    value=0,
    min=0,
    max=(p2_df.index.max() - p2_df.index.min()).days,
    step=1,
    description="Press play",
    disabled=False))

interactive(children=(Play(value=0, description='Press play', max=1478), Output()), _dom_classes=('widget-inte…

<function __main__.draw_graph(day)>

In [4]:
# This is the GIF version which is much smoother
# Some required libraries: 
#     pip install gif                 # Creates gifs
#     pip install "gif[altair]"       # Creates altair gifs
#     conda install -c conda-forge altair_saver  # Allows saving of altair images to PNG format (pip version requires other libs)
#     pip install pyderman            # Installs chromium driver necessary to save altair images

import gif
import pyderman as driver

driver.install(browser=driver.chrome, file_directory='./', filename='chromedriver.exe') #filename is important!

@gif.frame
def make_gif(day):
    return draw_graph(day)

frames = []
for i in range((p2_df.index.max() - p2_df.index.min()).days):
    frame = make_gif(i)
    frames.append(frame)

gif.save(frames, 'dt_full.gif', duration=50, unit="ms", between="frames")
# NOTE: dt_compressed.gif was created by uploading to Imgur and redownloading (creates a gifv)

	To:  d:\GitHub\Altair-Examples---Data-Visualizations\chromedriver_88.0.4324.96.zip
Download for 64 version failed; Trying alternates.
	To:  d:\GitHub\Altair-Examples---Data-Visualizations\chromedriver_88.0.4324.96.zip


![GIF of output](images/dt_compressed.gif)

Video also uploaded to [YouTube](https://youtu.be/ZUAd7FTFT9w)